In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np
from sklearn.preprocessing import StandardScaler


In [5]:
import warnings
warnings.filterwarnings("ignore")

In [203]:
data = pd.read_csv('facies_vectors.csv')
data = data.fillna(data['PE'].mean())

In [204]:
for well in set(data['Well Name']):
    print(well)
    print(set(data['Facies'][data['Well Name'] == well]))

NEWBY
{2, 3, 4, 5, 6, 7, 8, 9}
KIMZEY A
{1, 2, 3, 4, 5, 6, 7, 8, 9}
LUKE G U
{2, 3, 4, 5, 6, 7, 8}
Recruit F9
{9}
CHURCHMAN BIBLE
{1, 2, 3, 4, 5, 6, 7, 8, 9}
CROSS H CATTLE
{1, 2, 3, 4, 5, 6, 7, 8}
NOLAN
{1, 2, 3, 4, 5, 6, 7, 8}
SHANKLE
{1, 2, 3, 4, 5, 6, 7, 8}
SHRIMPLIN
{2, 3, 4, 5, 6, 7, 8, 9}
ALEXANDER D
{2, 3, 4, 5, 6, 7, 8, 9}


In [198]:
def find_diff(row, well):
    if len(prev_depth_features[well]) == 0:
        prev_depth_features[well] = row.values[4:]
        return
    diff = row.values[4:] - prev_depth_features[well]
    prev_depth_features[well] = row.values[4:]
    return diff
def check_changing_class(class_, well):
    if len(prev_class[well]) == 0:
        prev_class[well].append(class_)
        return
    if class_ == prev_class[well][-1]:
        prev_class[well].append(class_)
        return 0
    else:
        prev_class[well].append(class_)
        return 1
data_well = dict()
prev_depth_features = dict()
new_data = pd.DataFrame()
prev_class= dict()
for well in set(data['Well Name']):
    prev_depth_features[well] = []
    prev_class[well] = []
    data_well[well] = data[data['Well Name'] == well]
    data_well[well] = data_well[well].sort_values(by=['Depth'])
    data_well[well]['diff'] = data_well[well].apply(lambda row: find_diff(row, well), axis=1)
    data_well[well]['Change_class'] = data_well[well].apply(lambda row: check_changing_class(row['Facies'], well), axis=1)
    data_well[well] = data_well[well].dropna()
    data_well[well]['GR_diff'] = data_well[well].apply(lambda row: row['diff'][0], axis=1)
    data_well[well]['ILD_log10_diff'] = data_well[well].apply(lambda row: row['diff'][1], axis=1)
    data_well[well]['DeltaPHI_diff'] = data_well[well].apply(lambda row: row['diff'][2], axis=1)
    data_well[well]['PHIND_diff'] = data_well[well].apply(lambda row: row['diff'][3], axis=1)
    data_well[well]['PE_diff'] = data_well[well].apply(lambda row: row['diff'][4], axis=1)
    data_well[well]['NM_M_diff'] = data_well[well].apply(lambda row: row['diff'][5], axis=1)
    data_well[well]['RELPOS_diff'] = data_well[well].apply(lambda row: row['diff'][6], axis=1)
    new_data = pd.concat([new_data, data_well[well]])
    new_data = new_data.drop(['diff'], axis=1)

In [199]:
def accuracy(y_predict, y_test):
    truth = [y_predict[index] for index in range(len(y_predict)) if y_predict[index] == y_test[index]]
    return len(truth)/len(y_predict)

In [200]:
import xgboost as xgb
from xgboost.sklearn import  XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from keras.models import Sequential
from keras.layers import Dense
def classification_diff(clf):
    acc = 0
    mean_y_predict = 0
    for iter in range(10):
        clf.fit(X_diff_train, y_diff_train)
        y_predict = clf.predict(X_diff_test)
        acc += len([i for i in range(len(y_predict)) if y_predict[i] == y_diff_test[i] and y_diff_test[i] == 1])/len([i for i in range(len(y_diff_test)) if y_diff_test[i] == 1])
        mean_y_predict += y_predict
    acc /= 10
    mean_y_predict /= 10
    print('Diff_accuracy',acc)
    return [acc, [round(mean_y_predict[index]) for index in range(len(mean_y_predict))]]

def classification(clf):
    acc = 0
    for iter in range(10):
        clf.fit(X_train, y_train)
        y_predict = clf.predict(new_X_test)
        acc += accuracy(y_predict, new_y_test)
    acc /= 10
    print('Accuracy',acc)
    return acc

In [202]:
global_acc = 0
test = dict()
train = dict()
y_predict = dict()
feature_names = ['GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS','GR_diff', 'ILD_log10_diff', 'DeltaPHI_diff', 'PHIND_diff', 'PE_diff', 'NM_M_diff', 'RELPOS_diff']
for well in set(new_data['Well Name']):
    test[well] = new_data[new_data['Well Name'] == well]
    train[well] = new_data[new_data['Well Name'] != well]
    X_diff_train = train[well][feature_names].values 
    y_diff_train = train[well]['Change_class'].values 
    X_diff_test = test[well][feature_names].values 
    y_diff_test = test[well]['Change_class'].values 
    clf_xgb = XGBClassifier(learning_rate=0.12, max_depth=3,n_estimators=150)
    result = classification_diff(clf_xgb)
    global_acc += result[0]
    y_predict[well] = result[1]
    print(global_acc/len(set(data['Well Name'])))

Diff_accuracy 0.16666666666666669
0.01851851851851852
Diff_accuracy 0.1875
0.03935185185185185
Diff_accuracy 0.2363636363636364
0.06561447811447813
Diff_accuracy 0.08536585365853658
0.07509957296542663
Diff_accuracy 0.07954545454545456
0.08393795680381047
Diff_accuracy 0.07228915662650602
0.0919700853178667
Diff_accuracy 0.2553191489361702
0.12033887964410783
Diff_accuracy 0.24074074074074076
0.14708785083752346
Diff_accuracy 0.15294117647058822
0.16408131488981104


In [180]:
def combine(X, y):
    print(X)
    print(len(y))
    print(len([element[-1] for element in X]))
    new_X = []
    new_y = []
    first_el = 1
    new_element = []
    for row in range(len(X)):
        if first_el == 1:
            new_element = X[row]
            new_y.append(y[row])
            #print(new_element)
            first_el = 0
        else:
            if X[row][-1] == 0:
                new_element = [(new_element[index] + X[row][index])/2 for index in range(len(new_element))]
                #print(new_element)
            else:
                new_X.append(new_element[0:-1])
                #print(new_element[0:-1])
                #print('new', len(new_element))
                new_element = X[row]
                new_y.append(y[row])
    return np.array(new_X), np.array(new_y)
            
global_acc = 0
test = dict()
train = dict()
feature_names = ['GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS']
for well in set(new_data['Well Name']):
    test[well] = new_data[new_data['Well Name'] == well]
    train[well] = new_data[new_data['Well Name'] != well]
    X_train = train[well][feature_names].values 
    #X_train = np.concatenate((X_train, np.array([train[well]['Change_class'].values]).transpose()), axis=1)
    y_train = train[well]['Facies'].values 
    X_test = test[well][feature_names].values
    X_test = np.concatenate((X_test, np.array([y_predict[well]]).transpose()), axis=1)
    y_test = test[well]['Facies'].values
    new_X_test, new_y_test = combine(X_test, y_test) 
    clf_xgb = XGBClassifier(learning_rate=0.12, max_depth=3,n_estimators=150)
    global_acc += classification(clf_xgb)
    print(global_acc/len(set(data['Well Name'])))

[[83.74   0.688  9.7   ...  1.     0.977  0.   ]
 [83.19   0.664 10.1   ...  1.     0.953  0.   ]
 [80.44   0.648 10.1   ...  1.     0.93   0.   ]
 ...
 [47.87   0.797  0.7   ...  2.     0.27   0.   ]
 [49.34   0.763  2.3   ...  2.     0.258  0.   ]
 [59.88   0.721  3.2   ...  2.     0.247  0.   ]]
462
462


KeyboardInterrupt: 

In [ ]:
global_acc = 0
test = dict()
train = dict()
feature_names = ['GR', 'ILD_log10', 'DeltaPHI', 'PHIND', 'PE', 'NM_M', 'RELPOS','GR_diff', 'ILD_log10_diff', 'DeltaPHI_diff', 'PHIND_diff', 'PE_diff', 'NM_M_diff', 'RELPOS_diff']
for well in set(new_data['Well Name']):
    test[well] = new_data[new_data['Well Name'] == well]
    train[well] = new_data[new_data['Well Name'] != well]
    X_train = train[well][feature_names].values 
    X_train = np.concatenate((X_train, np.array([train[well]['Change_class'].values]).transpose()), axis=1)
    y_train = train[well]['Facies'].values 
    X_test = test[well][feature_names].values 
    X_test = np.concatenate((X_test, np.array([y_predict[well]]).transpose()), axis=1)
    y_test = test[well]['Facies'].values 
    clf_xgb = XGBClassifier(learning_rate=0.12, max_depth=3,n_estimators=150)
    global_acc += classification(clf_xgb)
    print(global_acc/len(set(data['Well Name'])))

In [52]:
clf_xgb = XGBClassifier(learning_rate=0.12, max_depth=3,n_estimators=150)
classification(clf_xgb)

With augm 0.5050100200400801
without anything 0.40718562874251496
With robust and aug 0.498997995991984
With robust 0.40718562874251496
With robust and aug and norm 0.5070140280561121
With robust and norm 0.40718562874251496


In [1]:
clf_log = LogisticRegression(multi_class='ovr')
classification(clf_log)


NameError: name 'LogisticRegression' is not defined

In [11]:
clf_per = MLPClassifier(hidden_layer_sizes=(100,100,100))
classification(clf_per)

With augm 0.5094967532467533
without anything 0.5349397590361445
With robust and aug 0.528814935064935
With robust 0.7278212525426381
With robust and aug and norm 0.5387175324675325
With robust and norm 0.7187713516403276


In [15]:
from keras.utils import to_categorical
y_train_tocat = to_categorical(y_train, 10)
y_test_tocat = to_categorical(y_test, 10)
model = Sequential()
model.add(Dense(100, input_dim=7, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train_robust_norm, y_train_tocat, validation_split=0.3, epochs=200, batch_size=10)
model.evaluate(X_test_robust_norm, y_test_tocat)

Train on 2032 samples, validate on 872 samples
Epoch 1/200
2032/2032 [==============================] - 1s 602us/step - loss: 1.4418 - acc: 0.4419 - val_loss: 1.1382 - val_acc: 0.5321
Epoch 2/200
2032/2032 [==============================] - 1s 401us/step - loss: 1.0839 - acc: 0.5595 - val_loss: 1.0559 - val_acc: 0.5757
Epoch 3/200
2032/2032 [==============================] - 1s 399us/step - loss: 1.0204 - acc: 0.5738 - val_loss: 1.0351 - val_acc: 0.5872
Epoch 4/200
2032/2032 [==============================] - 1s 414us/step - loss: 0.9801 - acc: 0.5984 - val_loss: 1.0218 - val_acc: 0.5883
Epoch 5/200
2032/2032 [==============================] - 1s 412us/step - loss: 0.9572 - acc: 0.6038 - val_loss: 0.9976 - val_acc: 0.5975
Epoch 6/200
2032/2032 [==============================] - 1s 405us/step - loss: 0.9268 - acc: 0.6147 - val_loss: 0.9828 - val_acc: 0.6170
Epoch 7/200
2032/2032 [==============================] - 1s 421us/step - loss: 0.9095 - acc: 0.6289 - val_loss: 0.9733 - val_acc: 0

2032/2032 [==============================] - 1s 414us/step - loss: 0.3608 - acc: 0.8469 - val_loss: 1.1219 - val_acc: 0.6640
Epoch 61/200
2032/2032 [==============================] - 1s 400us/step - loss: 0.3622 - acc: 0.8538 - val_loss: 1.1036 - val_acc: 0.6743
Epoch 62/200
2032/2032 [==============================] - 1s 409us/step - loss: 0.3439 - acc: 0.8647 - val_loss: 1.0935 - val_acc: 0.6766
Epoch 63/200
2032/2032 [==============================] - 1s 405us/step - loss: 0.3407 - acc: 0.8612 - val_loss: 1.1531 - val_acc: 0.6835
Epoch 64/200
2032/2032 [==============================] - 1s 399us/step - loss: 0.3356 - acc: 0.8602 - val_loss: 1.1496 - val_acc: 0.6823
Epoch 65/200
2032/2032 [==============================] - 1s 376us/step - loss: 0.3287 - acc: 0.8696 - val_loss: 1.1381 - val_acc: 0.6892
Epoch 66/200
2032/2032 [==============================] - 1s 402us/step - loss: 0.3317 - acc: 0.8745 - val_loss: 1.1298 - val_acc: 0.6881
Epoch 67/200
2032/2032 [=======================

2032/2032 [==============================] - 1s 400us/step - loss: 0.1074 - acc: 0.9675 - val_loss: 2.1094 - val_acc: 0.6789
Epoch 179/200
2032/2032 [==============================] - 1s 429us/step - loss: 0.1677 - acc: 0.9562 - val_loss: 2.1435 - val_acc: 0.6743
Epoch 180/200
2032/2032 [==============================] - 1s 412us/step - loss: 0.2293 - acc: 0.9291 - val_loss: 2.0666 - val_acc: 0.6904
Epoch 181/200
2032/2032 [==============================] - 1s 414us/step - loss: 0.1123 - acc: 0.9656 - val_loss: 2.1346 - val_acc: 0.6950
Epoch 182/200
2032/2032 [==============================] - 1s 409us/step - loss: 0.1107 - acc: 0.9636 - val_loss: 2.1475 - val_acc: 0.6709
Epoch 183/200
2032/2032 [==============================] - 1s 421us/step - loss: 0.0865 - acc: 0.9754 - val_loss: 2.0536 - val_acc: 0.6915
Epoch 184/200
2032/2032 [==============================] - 1s 411us/step - loss: 0.0681 - acc: 0.9808 - val_loss: 2.0516 - val_acc: 0.6858
Epoch 185/200
2032/2032 [================

[1.9645717413071169, 0.6899598386871767]

In [16]:
import pandas as pd
df = pd.DataFrame()
df['y1'] = [1, 2]
df['y2'] = [2, 3]
df

,y1,y2
0,1,2
1,2,3


In [45]:
import numpy as np
def add_std(y1, y2):
    return np.std([y1, y2]), np.mean([y1, y2]), min([y1, y2])
df['Result'] = df.apply(lambda row: add_std(row['y1'], row['y2']), axis=1)

In [50]:
df['std'] = df.apply(lambda row: row['Result'][0], axis = 1)
df['mean'] = df.apply(lambda row: row['Result'][1], axis = 1)
df['min'] = df.apply(lambda row: row['Result'][2], axis = 1)

In [56]:
df

,y1,y2,std,mean,Result,min
0,1,2,0.5,1.5,"(0.5, 1.5, 1.0)",1.0
1,2,3,0.5,2.5,"(0.5, 2.5, 2.0)",2.0


In [58]:
df = df.drop(['Result'], axis=1)

In [59]:
df

,y1,y2,std,mean,min
0,1,2,0.5,1.5,1.0
1,2,3,0.5,2.5,2.0


In [ ]:
acc= 0
    for iter in range(10):
        clf.fit(X_train, y_train)
        y_predict = clf.predict(X_test)
        acc += accuracy(y_predict, y_test)
    acc /= 10
    print('without anything',acc)
    acc= 0
    for iter in range(10):
        clf.fit(X_aug_train_robust, y_aug_train)
        y_predict = clf.predict(X_aug_test_robust)
        acc += accuracy(y_predict, y_aug_test)
    acc /= 10
    print('With robust and aug',acc)
    acc = 0
    for iter in range(10):
        clf.fit(X_train_robust, y_train)
        y_predict = clf.predict(X_test_robust)
        acc += accuracy(y_predict, y_test)
    acc /= 10
    print('With robust',acc)
    acc= 0
    for iter in range(10):
        clf.fit(X_aug_train_robust_norm, y_aug_train)
        y_predict = clf.predict(X_aug_test_robust_norm)
        acc += accuracy(y_predict, y_aug_test)
    acc /= 10
    print('With robust and aug and norm',acc)
    acc = 0
    for iter in range(10):
        clf.fit(X_train_robust_norm, y_train)
        y_predict = clf.predict(X_test_robust_norm)
        acc += accuracy(y_predict, y_test)
    acc /= 10
    print('With robust and norm',acc)

In [49]:
A = np.array([[1, 2, 3], [2, 3, 4]])
len(A)

2